In [3]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [1]:
#pulling from Oscars API information
from ipynb.fs.full.Oscar_information_3 import nominees_df

API request successful! Status code: 200
Number of nominees retrieved: 5
   id                 name  edition  year
0   1   1st Academy Awards        1  1927
1   2   2nd Academy Awards        2  1928
2   3   3rd Academy Awards        3  1929
3   4   4th Academy Awards        4  1930
4   5   5th Academy Awards        5  1931
5   6   6th Academy Awards        6  1932
6   7   7th Academy Awards        7  1934
7   8   8th Academy Awards        8  1935
8   9   9th Academy Awards        9  1936
9  10  10th Academy Awards       10  1937

    edition  year
0        64  1991
1        65  1992
2        66  1993
3        67  1994
4        68  1995
5        69  1996
6        70  1997
7        88  2015
8        89  2016
9        90  2017
10       91  2018
11       92  2019
12       93  2020
13       94  2021
Trying nominees URL for Actor In A Leading Role in 1991: https://theawards.vercel.app/api/oscars/editions/64/categories/4576/nominees
Trying nominees URL for Actress In A Leading Role in 1991: h

In [2]:
#all packages needed for project
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import time

In [3]:
#creating a list of all the urls for nominated movies and removing movies repeated due to multiple nominations
nominees_df_unique = nominees_df[['year', 'movie']].drop_duplicates()
movies = nominees_df_unique.values
#organizing urls in lists by year
url_91 = []
url_92 = []
url_93 = []
url_94 = []
url_95 = []
url_96 = []
url_97 = []
url_15 = []
url_16 = []
url_17 = []
url_18 = []
url_19 = []
url_20 = []
url_21 = []
#replaces the spaces in movie titles with "-" for urls
def titles(movie):
    for i in movie:
        return movie.replace(" ", "-")
#main loop that builds urls
for i in movies:
    year, movie_name = i
    url_ind = 'https://www.the-numbers.com/movie/'+titles(movie_name)+'#tab=box-office' #testing url
    if year == 1991:
        url_91.append(url_ind)
    elif year == 1992:
        url_92.append(url_ind)
    elif year == 1993:
        url_93.append(url_ind)
    elif year == 1994:
        url_94.append(url_ind)
    elif year == 1995:
        url_95.append(url_ind)
    elif year == 1996:
        url_96.append(url_ind)
    elif year == 1997:
        url_97.append(url_ind)
    elif year == 2015:
        url_15.append(url_ind)
    elif year == 2016:
        url_16.append(url_ind)
    elif year == 2017:
        url_17.append(url_ind)
    elif year == 2018:
        url_18.append(url_ind)
    elif year == 2019:
        url_19.append(url_ind)
    elif year == 2020:
        url_20.append(url_ind)
    elif year == 2021:
        url_21.append(url_ind)


    


In [22]:
#class that scrapes each individual url we listed, used with help of AI
class MonthNumberScraper:
    def __init__(self, urls, retries=3, delay=3):
        """
        urls: list of movie URLs (e.g., url_91, url_92, etc.)
        retries: number of fetch retries
        delay: delay between retries
        """
        self.urls = list(dict.fromkeys(urls))  # remove duplicates
        self.retries = retries
        self.delay = delay
        self.user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
            "Mozilla/5.0 (X11; Linux x86_64)",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)",
        ]

    def fetch_page(self, url):
        """Fetch page with retries and random user-agent."""
        for attempt in range(self.retries):
            try:
                headers = {"User-Agent": random.choice(self.user_agents)} #randomly chooses a user agent so server doesnt block you
                resp = requests.get(url, headers=headers, timeout=40) #try requesting url with 40-second timeout 
                resp.raise_for_status() #raising exception if there is an error
                return BeautifulSoup(resp.text, "html.parser")
            except Exception as e:
                if attempt == self.retries - 1:
                    raise
                time.sleep(self.delay)

    @staticmethod #method does not take self
    def find_weekend_table(soup): #finding the table with weekend box office data on "the Numbers" website
        """Find 8-column Weekend Box Office Performance table."""
        for h2 in soup.find_all("h2"):
            if "Weekend Box Office Performance" in h2.get_text():
                for tbl in h2.find_all_next("table"): #look at every table that comes after heading
                    ths = tbl.find_all("th")
                    if len(ths) == 8: #if the table already has 8 headers, it is probaly the correct table
                        return tbl
                    first_tr = tbl.find("tr")
                    if first_tr and len(first_tr.find_all("td")) == 8: # if the first row exists and has 8 <td> cells, it is the correct table
                        return tbl
        return None #if no table is found, return none

    @staticmethod
    def extract_table_to_df(table):
        """Convert table to DataFrame and clean numeric fields."""
        rows = [] #storing all of the rows of data
        for tr in table.find_all("tr"):
            if tr.find_all("th"): #skipping header rows that use <th>
                continue
            cells = [td.get_text(strip=True) for td in tr.find_all("td")]
            if len(cells) == 8: #only including rows that have excactly 8 cells
                rows.append(cells)
        if not rows: #if no valid rows were found
            return None
        #defining columns for DataFrame
        columns = ["Date", "Rank", "Gross", "% Change", "Theaters",
                   "Per Theater", "Total Gross", "Week"]
        df = pd.DataFrame(rows, columns=columns)

        #list of columns that have money value
        money_cols = ["Gross", "Per Theater", "Total Gross"]
        for c in money_cols:
            if c in df.columns:
                df[c] = pd.to_numeric(
                    df[c].str.replace(r"[^\d.-]", "", regex=True),
                    errors="coerce"
                )
    
        #pulling date column and extracting values
        if "Date" in df.columns:
            df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        #extracting the week part of the data and turning it into an integer
        if "Week" in df.columns:
            df["Week"] = pd.to_numeric(
                df["Week"].str.extract(r"(\d+)")[0],
                errors="coerce"
            ).astype("Int64")
    
        return df

    @staticmethod
    def clean_title_braces(title):
        """Remove {...} from movie title."""
        return re.sub(r"\{.*?\}", "", title).strip()

    @staticmethod
    def canonicalize_title(title):
        """Remove apostrophes, periods, normalize spaces into dashes."""
        t = title.replace("'", "").replace(".", "")
        t = re.sub(r"\s+", "-", t)
        t = re.sub(r"-{2,}", "-", t)
        return t.strip("-")

    @staticmethod
    def rotate_title(title):
        """Rotate first dash-separated segment to the end."""
        parts = title.split("-")
        if len(parts) > 1:
            return "-".join(parts[1:] + [parts[0]])
        return title

    def try_fallbacks(self, original_title, tried_urls): #for any urls that arent working
        """Try multiple title transformations until table found. Return soup, used_url, or None."""
        #cleaning original title
        clean = self.canonicalize_title(original_title)
        candidates = [clean]

        # handling "&" in titles 
        if "&" in clean or "and" not in clean:
            candidates.append(clean.replace("&", "and"))

        # generating a title backwards
        for cand in list(candidates):
            rotated = self.rotate_title(cand)
            if rotated not in candidates:
                candidates.append(rotated)
        #trying each method above to see which works for movie
        base_url_fmt = "https://www.the-numbers.com/movie/{title}#tab=box-office"
        for t in candidates:
            url_try = base_url_fmt.format(title=t)
            #skip any urls that have already been tried
            if url_try in tried_urls:
                continue
            #marking url as a tried url
            tried_urls.add(url_try)
            try:
                soup = self.fetch_page(url_try)
            except Exception:
                continue
            #checking to see if page contains a box office table
            table = self.find_weekend_table(soup)
            if table is not None:
                return soup, url_try
        # if no attempt work, return None
        return None, None  # all fallbacks failed

    def scrape_all(self):
        """Main loop to scrape all URLs in self.urls."""
        results = [] #list of all the final results from scraping

        for i, url in enumerate(self.urls, start=1): #looping through url with counter (i = index, url = url link)
            print(f"\n--- Scraping {i}/{len(self.urls)}: {url} ---")
            tried_urls = set() #keeping track of the urls we've already used
            original_title = url.split("/movie/")[-1].split("#")[0] #extracting the movie title from the url

            # Try original URL first
            try:
                soup = self.fetch_page(url)
                tried_urls.add(url)
                table = self.find_weekend_table(soup)
            except Exception:
                table = None

            # Fallbacks from try_fallbacks() method
            if table is None:
                soup, fallback_url = self.try_fallbacks(original_title, tried_urls)
                #if all of the fallbacks fail record failure and skip
                if soup is None:
                    print("All fallbacks failed. Using original URL.")
                    results.append({"title": original_title, "url": url, "data": None})
                    continue
                else:
                    #using successful fallback url and table
                    url = fallback_url
                    table = self.find_weekend_table(soup)
            #converting table into DataFrame
            df = self.extract_table_to_df(table)
            if df is None or df.empty:
                print("Table found but empty. Using original URL with no data.")
                results.append({"title": original_title, "url": url, "data": None})
                continue

            # Clean title from URL
            cleaned_title = self.clean_title_braces(original_title)
            results.append({"title": cleaned_title, "url": url, "data": df})

        return results
        
    def month_summary(self, results):
        summary_list = [] #list of each movie summary

        for movie_data in results:
            df = movie_data["data"]
            if df is not None and "Date" in df.columns:
                #Convert Date column into actual datetime
                df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    
                #storing first full date in weekend box office table
                first_date = df["Date"].min()
    
                #number in the year of the first month each movie was in theaters
                first_month = first_date.month if pd.notna(first_date) else None

                summary_list.append({
                    "movie": movie_data["title"],
                    "url": movie_data["url"],
                    "first_full_date_in_theaters": first_date, #returning full date of first week in theaters
                    "first_month_in_theaters": first_month #returning month number
                })

            else:
                summary_list.append({
                    "movie": movie_data["title"],
                    "url": movie_data["url"],
                    "first_full_date_in_theaters": None,
                    "first_month_in_theaters": None
                })

        return pd.DataFrame(summary_list)
        

In [23]:
#what is done here is done for each year
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_91) #creating instance per year
results = scraper.scrape_all() #scraping urls in list instance

summary_df = scraper.month_summary(results) #extracting date and month for each movie

print(summary_df) 

summary_df.to_csv("movie_months.csv", index=False) #saving all data in table in csv file


<class '__main__.MonthNumberScraper'>

--- Scraping 1/14: https://www.the-numbers.com/movie/Bugsy#tab=box-office ---

--- Scraping 2/14: https://www.the-numbers.com/movie/Cape-Fear#tab=box-office ---

--- Scraping 3/14: https://www.the-numbers.com/movie/The-Silence-of-the-Lambs#tab=box-office ---

--- Scraping 4/14: https://www.the-numbers.com/movie/The-Prince-of-Tides#tab=box-office ---

--- Scraping 5/14: https://www.the-numbers.com/movie/The-Fisher-King#tab=box-office ---

--- Scraping 6/14: https://www.the-numbers.com/movie/Thelma-&-Louise#tab=box-office ---

--- Scraping 7/14: https://www.the-numbers.com/movie/Rambling-Rose#tab=box-office ---

--- Scraping 8/14: https://www.the-numbers.com/movie/For-the-Boys#tab=box-office ---

--- Scraping 9/14: https://www.the-numbers.com/movie/Boyz-N-the-Hood#tab=box-office ---

--- Scraping 10/14: https://www.the-numbers.com/movie/JFK#tab=box-office ---

--- Scraping 11/14: https://www.the-numbers.com/movie/Beauty-and-the-Beast#tab=box-office 

In [24]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_92)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)


<class '__main__.MonthNumberScraper'>

--- Scraping 1/15: https://www.the-numbers.com/movie/Chaplin#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/Unforgiven#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/Scent-of-a-Woman#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/The-Crying-Game#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/Malcolm-X#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/Indochine#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Passion-Fish#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Love-Field#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 9/15: https://www.the-numbers.com/movie/Lorenzo's-Oil#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/Howards-End#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/The-Player#t

In [27]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_93)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/14: https://www.the-numbers.com/movie/In-the-Name-of-the-Father#tab=box-office ---

--- Scraping 2/14: https://www.the-numbers.com/movie/What's-Love-Got-to-Do-with-It#tab=box-office ---

--- Scraping 3/14: https://www.the-numbers.com/movie/Philadelphia#tab=box-office ---

--- Scraping 4/14: https://www.the-numbers.com/movie/The-Remains-of-the-Day#tab=box-office ---

--- Scraping 5/14: https://www.the-numbers.com/movie/Schindler's-List#tab=box-office ---

--- Scraping 6/14: https://www.the-numbers.com/movie/Six-Degrees-of-Separation#tab=box-office ---

--- Scraping 7/14: https://www.the-numbers.com/movie/The-Piano#tab=box-office ---

--- Scraping 8/14: https://www.the-numbers.com/movie/Shadowlands#tab=box-office ---

--- Scraping 9/14: https://www.the-numbers.com/movie/Short-Cuts#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 10/14: https://www.the-numbers.com/movie/The-Fugitive#tab=box-office ---

--- Scr

In [28]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_94)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/15: https://www.the-numbers.com/movie/The-Shawshank-Redemption#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/Forrest-Gump#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/The-Madness-of-King-George#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Nobody's-Fool#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/15: https://www.the-numbers.com/movie/Pulp-Fiction#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/15: https://www.the-numbers.com/movie/Nell#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Blue-Sky#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Tom-&-Viv#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Little-Women#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/The-Client#tab=box-office ---

--- Sc

In [29]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_95)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/14: https://www.the-numbers.com/movie/Leaving-Las-Vegas#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 2/14: https://www.the-numbers.com/movie/Mr.-Holland's-Opus#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/14: https://www.the-numbers.com/movie/Nixon#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:28: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(resp.text, "html.parser")


All fallbacks failed. Using original URL.

--- Scraping 4/14: https://www.the-numbers.com/movie/Dead-Man-Walking#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/14: https://www.the-numbers.com/movie/The-Postman-(Il-Postino)#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/14: https://www.the-numbers.com/movie/Casino#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/14: https://www.the-numbers.com/movie/The-Bridges-of-Madison-County#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/14: https://www.the-numbers.com/movie/Sense-and-Sensibility#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 9/14: https://www.the-numbers.com/movie/Babe#tab=box-office ---

--- Scraping 10/14: https://www.the-numbers.com/movie/Braveheart#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 11/14: https://www.the-numbers.com/movie/Apollo-13#tab=box-offi

In [30]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_96)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/13: https://www.the-numbers.com/movie/Jerry-Maguire#tab=box-office ---

--- Scraping 2/13: https://www.the-numbers.com/movie/The-English-Patient#tab=box-office ---

--- Scraping 3/13: https://www.the-numbers.com/movie/The-People-vs.-Larry-Flynt#tab=box-office ---

--- Scraping 4/13: https://www.the-numbers.com/movie/Shine#tab=box-office ---

--- Scraping 5/13: https://www.the-numbers.com/movie/Sling-Blade#tab=box-office ---

--- Scraping 6/13: https://www.the-numbers.com/movie/Secrets-&-Lies#tab=box-office ---

--- Scraping 7/13: https://www.the-numbers.com/movie/Marvin's-Room#tab=box-office ---

--- Scraping 8/13: https://www.the-numbers.com/movie/Fargo#tab=box-office ---

--- Scraping 9/13: https://www.the-numbers.com/movie/Breaking-the-Waves#tab=box-office ---

--- Scraping 10/13: https://www.the-numbers.com/movie/The-Crucible#tab=box-office ---

--- Scraping 11/13: https://www.the-numbers.com/movie/Hamlet#tab=box-office ---
All 

In [31]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_97)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/15: https://www.the-numbers.com/movie/Good-Will-Hunting#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/The-Apostle#tab=box-office ---

--- Scraping 3/15: https://www.the-numbers.com/movie/Ulee's-Gold#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Wag-the-Dog#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:28: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(resp.text, "html.parser")


All fallbacks failed. Using original URL.

--- Scraping 5/15: https://www.the-numbers.com/movie/As-Good-as-It-Gets#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/The-Wings-of-the-Dove#tab=box-office ---

--- Scraping 7/15: https://www.the-numbers.com/movie/Afterglow#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Mrs.-Brown#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Titanic#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/The-Full-Monty#tab=box-office ---

--- Scraping 11/15: https://www.the-numbers.com/movie/L.A.-Confidential#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 12/15: https://www.the-numbers.com/movie/The-Sweet-Hereafter#tab=box-office ---

--- Scraping 13/15: https://www.the-numbers.com/movie/Donnie-Brasco#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 14/15: https://www.the-numbers.com/movie/Boogie-Nights#tab=box

In [32]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_15)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/17: https://www.the-numbers.com/movie/Trumbo#tab=box-office ---

--- Scraping 2/17: https://www.the-numbers.com/movie/The-Martian#tab=box-office ---

--- Scraping 3/17: https://www.the-numbers.com/movie/The-Revenant#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 4/17: https://www.the-numbers.com/movie/Steve-Jobs#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/17: https://www.the-numbers.com/movie/The-Danish-Girl#tab=box-office ---

--- Scraping 6/17: https://www.the-numbers.com/movie/Carol#tab=box-office ---

--- Scraping 7/17: https://www.the-numbers.com/movie/Room#tab=box-office ---

--- Scraping 8/17: https://www.the-numbers.com/movie/Joy#tab=box-office ---

--- Scraping 9/17: https://www.the-numbers.com/movie/45-Years#tab=box-office ---

--- Scraping 10/17: https://www.the-numbers.com/movie/Brooklyn#tab=box-office ---

--- Scraping 11/17: https://www.the-numbers.com/movie/

In [33]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_16)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)
                                                 
print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/16: https://www.the-numbers.com/movie/Manchester-by-the-Sea#tab=box-office ---

--- Scraping 2/16: https://www.the-numbers.com/movie/Hacksaw-Ridge#tab=box-office ---

--- Scraping 3/16: https://www.the-numbers.com/movie/La-La-Land#tab=box-office ---

--- Scraping 4/16: https://www.the-numbers.com/movie/Captain-Fantastic#tab=box-office ---

--- Scraping 5/16: https://www.the-numbers.com/movie/Fences#tab=box-office ---

--- Scraping 6/16: https://www.the-numbers.com/movie/Elle#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/16: https://www.the-numbers.com/movie/Loving#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/16: https://www.the-numbers.com/movie/Jackie#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 9/16: https://www.the-numbers.com/movie/Florence-Foster-Jenkins#tab=box-office ---

--- Scraping 10/16: https://www.the-numbers.com/movie/Arrival#tab=box-office ---

--- Scraping 11/16: https://www.the-numbers.com/movie/Moonlight#tab=box-office ---

--- Scraping 12/16: https://www.the-numbers.com/movie/Hell-or-High-Water#tab=box-office ---

--- Scraping 13/16: https://www.the-numbers.com/movie/Hidden-Figures#tab=box-office ---

--- Scraping 14/16: https://www.the-numbers.com/movie/Lion#tab=box-office ---

--- Scraping 15/16: https://www.the-numbers.com/movie/The-Lobster#tab=box-office ---

--- Scraping 16/16: https://www.the-numbers.com/movie/20th-Century-Women#tab=box-office ---
                      movie  \
0     Manchester-by-the-Sea   
1             Hacksaw-Ridge   
2                La-La-Land   
3         Captain-Fantastic   
4                    Fences   
5                      Elle   
6                    Loving   
7                    Jackie   
8   Florence-Foste

In [34]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_17)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/16: https://www.the-numbers.com/movie/Call-Me-by-Your-Name#tab=box-office ---

--- Scraping 2/16: https://www.the-numbers.com/movie/Phantom-Thread#tab=box-office ---

--- Scraping 3/16: https://www.the-numbers.com/movie/Get-Out#tab=box-office ---

--- Scraping 4/16: https://www.the-numbers.com/movie/Darkest-Hour#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/16: https://www.the-numbers.com/movie/Roman-J.-Israel,-Esq.#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/16: https://www.the-numbers.com/movie/The-Shape-of-Water#tab=box-office ---

--- Scraping 7/16: https://www.the-numbers.com/movie/Three-Billboards-outside-Ebbing,-Missouri#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/16: https://www.the-numbers.com/movie/I,-Tonya#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 9/16: https://www.the-numbers.com/movie/Lady-Bi

In [35]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_18)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/15: https://www.the-numbers.com/movie/Vice#tab=box-office ---

--- Scraping 2/15: https://www.the-numbers.com/movie/A-Star-Is-Born#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/15: https://www.the-numbers.com/movie/At-Eternity's-Gate#tab=box-office ---

--- Scraping 4/15: https://www.the-numbers.com/movie/Bohemian-Rhapsody#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/15: https://www.the-numbers.com/movie/Green-Book#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/15: https://www.the-numbers.com/movie/Roma#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/15: https://www.the-numbers.com/movie/The-Wife#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/15: https://www.the-numbers.com/movie/The-Favourite#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 9/15: https://ww

C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 12/15: https://www.the-numbers.com/movie/Black-Panther#tab=box-office ---

--- Scraping 13/15: https://www.the-numbers.com/movie/The-Ballad-of-Buster-Scruggs#tab=box-office ---

--- Scraping 14/15: https://www.the-numbers.com/movie/If-Beale-Street-Could-Talk#tab=box-office ---

--- Scraping 15/15: https://www.the-numbers.com/movie/First-Reformed#tab=box-office ---
                           movie  \
0                           Vice   
1                 A-Star-Is-Born   
2             At-Eternity's-Gate   
3              Bohemian-Rhapsody   
4                     Green-Book   
5                           Roma   
6                       The-Wife   
7                  The-Favourite   
8       Can-You-Ever-Forgive-Me?   
9                 BlacKkKlansman   
10                      Cold-War   
11                 Black-Panther   
12  The-Ballad-of-Buster-Scruggs   
13    If-Beale-Street-Could-Talk   
14                First-Reformed   

                                          

In [36]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_19)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False) 

<class '__main__.MonthNumberScraper'>

--- Scraping 1/15: https://www.the-numbers.com/movie/Pain-and-Glory#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 2/15: https://www.the-numbers.com/movie/Once-upon-a-Time...in-Hollywood#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/15: https://www.the-numbers.com/movie/Marriage-Story#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 4/15: https://www.the-numbers.com/movie/Joker#tab=box-office ---

--- Scraping 5/15: https://www.the-numbers.com/movie/The-Two-Popes#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/15: https://www.the-numbers.com/movie/Harriet#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/15: https://www.the-numbers.com/movie/Little-Women#tab=box-office ---

--- Scraping 8/15: https://www.the-numbers.com/movie/Bombshell#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Judy#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 10/15: https://www.the-numbers.com/movie/The-Irishman#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 11/15: https://www.the-numbers.com/movie/1917#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 12/15: https://www.the-numbers.com/movie/Parasite#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 13/15: https://www.the-numbers.com/movie/Ford-v-Ferrari#tab=box-office ---

--- Scraping 14/15: https://www.the-numbers.com/movie/Jojo-Rabbit#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 15/15: https://www.the-numbers.com/movie/Knives-Out#tab=box-office ---
All fallbacks failed. Using original URL.
                              movie  \
0                    Pain-and-Glory   
1   Once-upon-a-Time...in-Hollywood   
2                    Marriage-Story   
3                             Joker   
4                     The-Two-Popes   
5                           Harriet   
6                      Little-Women   
7                         Bombshell   
8                              Judy   
9                   

In [37]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_20)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/15: https://www.the-numbers.com/movie/Sound-of-Metal#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 2/15: https://www.the-numbers.com/movie/Ma-Rainey's-Black-Bottom#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/15: https://www.the-numbers.com/movie/The-Father#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 4/15: https://www.the-numbers.com/movie/Mank#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 5/15: https://www.the-numbers.com/movie/Minari#tab=box-office ---

--- Scraping 6/15: https://www.the-numbers.com/movie/The-United-States-vs.-Billie-Holiday#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 7/15: https://www.the-numbers.com/movie/Pieces-of-a-Woman#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 8/15: https://www.the-numbers.com/movie/Nomadland#tab=box-office ---

--- Scraping 9/15: https://www.the-numbers.com/movie/Promising-Young-Woman#tab=box-office ---

--- Scraping 10/15: https://www.the-numbers.com/movie/Another-Round#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 11/15: https://www.the-numbers.com/movie/Judas-and-the-Black-Messiah#tab=box-office ---

--- Scraping 12/15: https://www.the-numbers.com/movie/The-Trial-of-the-Chicago-7#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 13/15: https://www.the-numbers.com/movie/Borat-Subsequent-Moviefilm:-Delivery-of-Prodigious-Bribe-to-American-Regime-for-Make-Benefit-Once-Glorious-Nation-of-Kazakhstan#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 14/15: https://www.the-numbers.com/movie/One-Night-in-Miami...#tab=box-office ---

--- Scraping 15/15: https://www.the-numbers.com/movie/The-White-Tiger#tab=box-office ---
All fallbacks failed. Using original URL.
                                                movie  \
0                                      Sound-of-Metal   
1                            Ma-Rainey's-Black-Bottom   
2                                          The-Father   
3                                                Mank   
4                                              Minari   
5                The-United-States-vs.-Billie-Holiday   
6                                   Pieces-of-a-Woman   
7                                           Nomadland   
8              

In [38]:
print(MonthNumberScraper)
scraper = MonthNumberScraper(url_21)
results = scraper.scrape_all()

summary_df = scraper.month_summary(results)

print(summary_df)

summary_df.to_csv("movie_months.csv", mode = "a", header=False, index=False)

<class '__main__.MonthNumberScraper'>

--- Scraping 1/18: https://www.the-numbers.com/movie/Being-the-Ricardos#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 2/18: https://www.the-numbers.com/movie/The-Power-of-the-Dog#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 3/18: https://www.the-numbers.com/movie/tick,-tick...BOOM!#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 4/18: https://www.the-numbers.com/movie/King-Richard#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 5/18: https://www.the-numbers.com/movie/The-Tragedy-of-Macbeth#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 6/18: https://www.the-numbers.com/movie/The-Eyes-of-Tammy-Faye#tab=box-office ---

--- Scraping 7/18: https://www.the-numbers.com/movie/The-Lost-Daughter#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 8/18: https://www.the-numbers.com/movie/Parallel-Mothers#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 9/18: https://www.the-numbers.com/movie/Spencer#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 10/18: https://www.the-numbers.com/movie/Belfast#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 11/18: https://www.the-numbers.com/movie/Drive-My-Car#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 12/18: https://www.the-numbers.com/movie/Licorice-Pizza#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 13/18: https://www.the-numbers.com/movie/West-Side-Story#tab=box-office ---


C:\Users\anaya\AppData\Local\Temp\ipykernel_31612\650165263.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")



--- Scraping 14/18: https://www.the-numbers.com/movie/CODA#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 15/18: https://www.the-numbers.com/movie/Don't-Look-Up#tab=box-office ---
All fallbacks failed. Using original URL.

--- Scraping 16/18: https://www.the-numbers.com/movie/Dune#tab=box-office ---

--- Scraping 17/18: https://www.the-numbers.com/movie/Nightmare-Alley#tab=box-office ---

--- Scraping 18/18: https://www.the-numbers.com/movie/The-Worst-Person-in-the-World#tab=box-office ---
All fallbacks failed. Using original URL.
                            movie  \
0              Being-the-Ricardos   
1            The-Power-of-the-Dog   
2              tick,-tick...BOOM!   
3                    King-Richard   
4          The-Tragedy-of-Macbeth   
5          The-Eyes-of-Tammy-Faye   
6               The-Lost-Daughter   
7                Parallel-Mothers   
8                         Spencer   
9                         Belfast   
10                   Drive-